In [34]:
import json
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import gaussian_kde

In [35]:
# import json
ticker1 = 'LULU'
ticker2 = 'FRCOY'

In [36]:
# Step 1: Retrieve data for both stocks
start_date = '2000-01-01'
end_date = '2024-01-01'

# Download data for each ticker
data1 = yf.download(ticker1, start=start_date, end=end_date, interval='1wk')
data1['Return'] = data1['Adj Close'].pct_change().dropna()

data2 = yf.download(ticker2, start=start_date, end=end_date, interval='1wk')
data2['Return'] = data2['Adj Close'].pct_change().dropna()

# Extract returns
returns_stock1 = data1['Return'].dropna()
returns_stock2 = data2['Return'].dropna()

# Step 2: KDE for each stock's returns
kde1 = gaussian_kde(returns_stock1)
kde2 = gaussian_kde(returns_stock2)

# Create x-axis range for KDE
x_grid = np.linspace(min(returns_stock1.min(), returns_stock2.min()),
                     max(returns_stock1.max(), returns_stock2.max()), 1000)

pdf_stock1 = kde1(x_grid)
pdf_stock2 = kde2(x_grid)

# Step 3: CDF Calculation
cdf_stock1 = np.cumsum(pdf_stock1) * (x_grid[1] - x_grid[0])
cdf_stock2 = np.cumsum(pdf_stock2) * (x_grid[1] - x_grid[0])

# Step 4: ICDF Calculation (integrated CDF)
icdf_stock1 = np.cumsum(cdf_stock1) * (x_grid[1] - x_grid[0])
icdf_stock2 = np.cumsum(cdf_stock2) * (x_grid[1] - x_grid[0])

# Step 5: Save data as JSON for use in Svelte
output_data = {
    "x_grid": x_grid.tolist(),
    "pdf_stock1": pdf_stock1.tolist(),
    "pdf_stock2": pdf_stock2.tolist(),
    "cdf_stock1": cdf_stock1.tolist(),
    "cdf_stock2": cdf_stock2.tolist(),
    "icdf_stock1": icdf_stock1.tolist(),
    "icdf_stock2": icdf_stock2.tolist(),
}

# Write to JSON file
with open('distribution_data.json', 'w') as f:
    json.dump(output_data, f)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [37]:
# import json

# use ticker 1 versus ticker 2 as file name
data_name = f'{ticker1}_vs_{ticker2}.json' 
file_path = f'/Users/victoriashi/GithubProject/info_viz/src/data/{data_name}'
# save data to file
with open(file_path, 'w') as f:
    json.dump(output_data, f)

## Get data for multiple stock pairs

In [48]:
stock_pairs = [
    # ('META', 'GOOGL'),   # Meta vs Google
    # ("LULU", "FRCOY"), # Burberry vs Fast Retailing
    ("DIS", "IQ"),        # Disney vs Six Flags
    ('BABA', 'AMZN'),    # Alibaba vs Amazon
    ('JD', 'WMT'),       # JD.com vs Walmart
    ('DASH', 'UPS'),    # DoorDash vs Grubhub
    ('TSLA', 'UAL'),    # Tesla vs United Airlines
]

In [49]:
# Loop through each stock pair
for ticker1, ticker2 in stock_pairs:
    data1 = yf.download(ticker1, start=start_date, end=end_date, interval='1wk')
    data1['Return'] = data1['Adj Close'].pct_change().dropna()
    
    data2 = yf.download(ticker2, start=start_date, end=end_date, interval='1wk')
    data2['Return'] = data2['Adj Close'].pct_change().dropna()
    
    returns_stock1 = data1['Return'].dropna()
    returns_stock2 = data2['Return'].dropna()
        
    kde1 = gaussian_kde(returns_stock1)
    kde2 = gaussian_kde(returns_stock2)
    
    x_grid = np.linspace(min(returns_stock1.min(), returns_stock2.min()),
                            max(returns_stock1.max(), returns_stock2.max()), 1000)
    
    pdf_stock1 = kde1(x_grid)
    pdf_stock2 = kde2(x_grid)
    
    cdf_stock1 = np.cumsum(pdf_stock1) * (x_grid[1] - x_grid[0])
    
    cdf_stock2 = np.cumsum(pdf_stock2) * (x_grid[1] - x_grid[0])
        
    icdf_stock1 = np.cumsum(cdf_stock1) * (x_grid[1] - x_grid[0])
    icdf_stock2 = np.cumsum(cdf_stock2) * (x_grid[1] - x_grid[0])
    
    output_data = {
        "x_grid": x_grid.tolist(),
        "pdf_stock1": pdf_stock1.tolist(),
        "pdf_stock2": pdf_stock2.tolist(),
        "cdf_stock1": cdf_stock1.tolist(),
        "cdf_stock2": cdf_stock2.tolist(),
        "icdf_stock1": icdf_stock1.tolist(),
        "icdf_stock2": icdf_stock2.tolist(),
    }
    
    
    data_name = f'{ticker1}_vs_{ticker2}.json'
    file_path = f'/Users/victoriashi/GithubProject/info_viz/src/data/{data_name}'
    with open(file_path, 'w') as f:
        json.dump(output_data, f)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
